In [1]:
try:
    import google.colab
    on_colab = True

    !gdown --id 11xL65BOrEc93TFYhCp7VmrEh2MWBNgwf -O depth.zip
    !unzip depth.zip -d ./depth

    !gdown --id 1l5xwvj3YZgKIDWSoE8Geom_FT9rSbKpn -O depth_edge.zip
    !unzip depth_edge.zip -d ./depth_edges

    from google.colab import  drive
    drive.mount('/content/drive')

except ImportError:
    on_colab = False

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=11xL65BOrEc93TFYhCp7VmrEh2MWBNgwf
To: /content/depth.zip
100% 16.6M/16.6M [00:00<00:00, 47.4MB/s]
Archive:  depth.zip
  inflating: ./depth/00000001.jpg    
  inflating: ./depth/00000002.jpg    
  inflating: ./depth/00000003.jpg    
  inflating: ./depth/00000004.jpg    
  inflating: ./depth/00000005.jpg    
  inflating: ./depth/00000006.jpg    
  inflating: ./depth/00000007.jpg    
  inflating: ./depth/00000008.jpg    
  inflating: ./depth/00000009.jpg    
  inflating: ./depth/00000010.jpg    
  inflating: ./depth/00000011.jpg    
  inflating: ./depth/00000012.jpg    
  inflating: ./depth/00000013.jpg    
  inflating: ./depth/00000014.jpg    
  inflating: ./depth/00000015.jpg    
  inflating: ./depth/00000016.jpg    
 

In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

In [9]:
!rm -r depth_pattern_analysis/

In [10]:
if on_colab:
    !git clone https://github.com/hamidrezafahimi/depth_pattern_analysis.git

Cloning into 'depth_pattern_analysis'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 282 (delta 113), reused 255 (delta 86), pack-reused 0 (from 0)
Receiving objects: 100% (282/282), 1.17 MiB | 9.04 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [2]:
on_colab = True

In [4]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 17.5 MB/s eta 0:00:00


In [3]:
import os
import sys

# Directories for depth images and template masks.
if on_colab:
    sys.path.append('/content/depth_pattern_analysis/gt_generation')
    depth_dir = "/content/depth"
    tmask_dir = "/content/depth_edges"
    output_dir = "/content/drive/MyDrive/outs5"
else:
    depth_dir = "../data/depth"
    tmask_dir = "../data/depth_edge"
    output_dir = "../data/outs"

# Get sorted list of filenames.
depth_files = sorted([f for f in os.listdir(depth_dir) if os.path.isfile(os.path.join(depth_dir, f))])
tmask_files = sorted([f for f in os.listdir(tmask_dir) if os.path.isfile(os.path.join(tmask_dir, f))])

# Check that both directories have exactly the same filenames.
if set(depth_files) != set(tmask_files):
    raise ValueError("Depth images and template masks must have matching filenames.")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

from search_threshs import AutoDepthBGSubtractor
# Create a ThresholdSearcher object with desired parameters and logging flags.
searcher = AutoDepthBGSubtractor()

In [ ]:
import cv2
import time
# Process each matching pair.
k = 0
for fname in depth_files:
    k += 1
    if k < 855:
        continue
    depth_path = os.path.join(depth_dir, fname)
    tmask_path = os.path.join(tmask_dir, fname)

    depth_image = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
    if depth_image is None:
        raise IOError(f"Could not load depth image: {depth_path}")
    template_mask = cv2.imread(tmask_path, cv2.IMREAD_GRAYSCALE)
    if template_mask is None:
        raise IOError(f"Could not load template mask: {tmask_path}")
    if depth_image.shape != template_mask.shape:
        raise ValueError(f"Image shape mismatch for {fname}")

    t0 = time.time()
    searcher.search(template_mask, depth_image, fname[:-4], output_dir)
    print("time: ", time.time() - t0)
    print("-------------")

Initial Mosaic Array:
 [[-1 -2 -2 -2 -1]
 [-1 -1 -2 -2 -1]
 [-1 -1 -2 -1 -1]
 [-1 -1 -1 -1 -1]]
iterate time:  10.747098922729492
Mosaic after assignment:
 [[ -1  -2  48   0  -1]
 [ -1  -1   2 124  -1]
 [ -1  -1  39  -1  -1]
 [ -1  -1  -1  -1  -1]]
Mosaic after adjacency fix:
 [[ -1  -2  48   0  -1]
 [ -1  -1   2 330  -1]
 [ -1  -1  39  -1  -1]
 [ -1  -1  -1  -1  -1]]
Saved final => /content/drive/MyDrive/outs5/00000855_bgsMasked.jpg
time:  11.914045810699463
-------------
Initial Mosaic Array:
 [[-1 -2 -1 -2 -1]
 [-1 -1 -2 -2 -1]
 [-1 -1 -2 -1 -1]
 [-1 -1 -1 -1 -1]]
iterate time:  11.105895757675171
Mosaic after assignment:
 [[ -1  29  -1   3  -1]
 [ -1  -1  11 492  -1]
 [ -1  -1  23  -1  -1]
 [ -1  -1  -1  -1  -1]]
Mosaic after adjacency fix:
 [[ -1  29  -1   3  -1]
 [ -1  -1  11 492  -1]
 [ -1  -1  23  -1  -1]
 [ -1  -1  -1  -1  -1]]
Saved final => /content/drive/MyDrive/outs5/00000856_bgsMasked.jpg
time:  12.29374361038208
-------------
Initial Mosaic Array:
 [[-2 -2 -1 -2 -1]
 [-1